In [1]:
# lets import the basic libraries
import numpy as np
import pandas as pd

# for data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# for jupyter notebook widgets
import ipywidgets as widgets
from ipywidgets import interact
from ipywidgets import interact_manual

# for interactive shells
from IPython.display import display

# supress warnings
import warnings
warnings.filterwarnings('ignore')

# setting upt the chart size and background
plt.rcParams['figure.figsize'] = (16, 5)
plt.style.use('fivethirtyeight')

In [2]:
# lets read the dataset
data = pd.read_csv('movie_metadata.csv')

In [3]:
# lets check the shape
print(data.shape)

(5043, 28)


In [4]:
# lets check the column wise info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   color                      5024 non-null   object 
 1   director_name              4939 non-null   object 
 2   num_critic_for_reviews     4993 non-null   float64
 3   duration                   5028 non-null   float64
 4   director_facebook_likes    4939 non-null   float64
 5   actor_3_facebook_likes     5020 non-null   float64
 6   actor_2_name               5030 non-null   object 
 7   actor_1_facebook_likes     5036 non-null   float64
 8   gross                      4159 non-null   float64
 9   genres                     5043 non-null   object 
 10  actor_1_name               5036 non-null   object 
 11  movie_title                5043 non-null   object 
 12  num_voted_users            5043 non-null   int64  
 13  cast_total_facebook_likes  5043 non-null   int64

In [6]:
# lets remove unnecessary columns from the dataset

# Use the 'drop()' function to drop the unnecessary columns

data = data.drop(['color',
                  'director_facebook_likes',
                  'actor_3_facebook_likes',
                  'actor_1_facebook_likes',
                  'cast_total_facebook_likes',
                  'actor_2_facebook_likes',
                  'facenumber_in_poster',
                  'content_rating',
                  'country',
                  'movie_imdb_link',
                  'aspect_ratio',
                  'plot_keywords'],
                   axis = 1)
data.columns

Index(['director_name', 'num_critic_for_reviews', 'duration', 'actor_2_name',
       'gross', 'genres', 'actor_1_name', 'movie_title', 'num_voted_users',
       'actor_3_name', 'num_user_for_reviews', 'language', 'budget',
       'title_year', 'imdb_score', 'movie_facebook_likes'],
      dtype='object')

In [7]:
# lets check the rows having high percentage of missing values in the dataset

round(100*(data.isnull().sum()/len(data.index)), 2)

director_name              2.06
num_critic_for_reviews     0.99
duration                   0.30
actor_2_name               0.26
gross                     17.53
genres                     0.00
actor_1_name               0.14
movie_title                0.00
num_voted_users            0.00
actor_3_name               0.46
num_user_for_reviews       0.42
language                   0.24
budget                     9.76
title_year                 2.14
imdb_score                 0.00
movie_facebook_likes       0.00
dtype: float64

In [8]:
# since 'gross' and 'budget' columns have large number of NaN values, drop all the rows with NaNs at this column using the 
# 'isnan' function of NumPy alongwith a negation '~'

data = data[~np.isnan(data['gross'])]
data = data[~np.isnan(data['budget'])]

In [9]:
# Now lets agin check the Missing Values column wise

data.isnull().sum()

director_name              0
num_critic_for_reviews     1
duration                   1
actor_2_name               5
gross                      0
genres                     0
actor_1_name               3
movie_title                0
num_voted_users            0
actor_3_name              10
num_user_for_reviews       0
language                   3
budget                     0
title_year                 0
imdb_score                 0
movie_facebook_likes       0
dtype: int64

In [10]:
# The rows for which the sum of Null is less than two are retained

data = data[data.isnull().sum(axis=1) <= 2]
data.isnull().sum()

director_name             0
num_critic_for_reviews    1
duration                  1
actor_2_name              2
gross                     0
genres                    0
actor_1_name              0
movie_title               0
num_voted_users           0
actor_3_name              7
num_user_for_reviews      0
language                  3
budget                    0
title_year                0
imdb_score                0
movie_facebook_likes      0
dtype: int64

In [11]:
# Lets impute the missing values

# using mean for numerical columns 
data['num_critic_for_reviews'].fillna(data['num_critic_for_reviews'].mean(), inplace = True)
data['duration'].fillna(data['duration'].mean(), inplace = True)

# using mode for categorical column
data['language'].fillna(data['language'].mode()[0], inplace = True)

# As we know that we cannot use statistical values for imputing the missing values of actor names, so we will replace the 
# actor names with "Unknown Actor"

data['actor_2_name'].fillna('Unknown Actor', inplace = True)
data['actor_3_name'].fillna('Unknown Actor', inplace = True)

# as we imputed all the missing values lets check the no. of total missing values in the dataset
data.isnull().sum().sum()

0

In [12]:
# Lets convert the gross and budget from $ to Million $ to make our analysis easier

data['gross'] = data['gross']/1000000
data['budget'] = data['budget']/1000000

In [13]:
# Lets create a profit column using the Budget and Gross

data['profit'] = data['gross'] - data['budget']

In [14]:
# Lets also check the name of top 10 Profitabel Movie

data[['profit','movie_title']].sort_values(by = 'profit', ascending = False).head(10)

,profit,movie_title
0,523.505847,Avatar
29,502.177271,Jurassic World
26,458.672302,Titanic
3024,449.935665,Star Wars: Episode IV - A New Hope
3080,424.449459,E.T. the Extra-Terrestrial
794,403.279547,The Avengers
17,403.279547,The Avengers
509,377.783777,The Lion King
240,359.544677,Star Wars: Episode I - The Phantom Menace
66,348.316061,The Dark Knight


In [15]:
# By lokking at the above result we can easily analyze that there are some duplicate

# Lets print the no. of rows before removing Duplicates
print("No. of Rows Before Removing Duplicates: ",data.shape[0])

# So lets remove all the duplicates from the data
data.drop_duplicates(subset = None, keep = 'first', inplace = True)

# Lets print the no. rows ofter removing Duplicates
print("No. of Rows After Removing Duplictes: ",data.shape[0])

No. of Rows Before Removing Duplicates:  3888
No. of Rows After Removing Duplictes:  3853


In [16]:
# Lets also check the name of top 10 Profitabel Movie

data[['movie_title','profit']].sort_values(by = 'profit', ascending = False).head(10)

,movie_title,profit
0,Avatar,523.505847
29,Jurassic World,502.177271
26,Titanic,458.672302
3024,Star Wars: Episode IV - A New Hope,449.935665
3080,E.T. the Extra-Terrestrial,424.449459
17,The Avengers,403.279547
509,The Lion King,377.783777
240,Star Wars: Episode I - The Phantom Menace,359.544677
66,The Dark Knight,348.316061
439,The Hunger Games,329.999255


In [17]:
# Lets check the values in the language column

data['language'].value_counts()

English       3673
French          37
Spanish         26
Mandarin        14
German          13
Japanese        12
Hindi           10
Cantonese        8
Italian          7
Korean           5
Portuguese       5
Norwegian        4
Thai             3
Danish           3
Hebrew           3
Dutch            3
Persian          3
Dari             2
Aboriginal       2
Indonesian       2
Hungarian        1
Zulu             1
Mongolian        1
Bosnian          1
Kazakh           1
Telugu           1
Romanian         1
Dzongkha         1
Swedish          1
Czech            1
None             1
Arabic           1
Russian          1
Vietnamese       1
Filipino         1
Aramaic          1
Icelandic        1
Maya             1
Name: language, dtype: int64

In [18]:
# Looking at the above output we can easily observe that out of 3,500 movies only 150 movies are of other language

# So it is better to keep only two languages that is English and foreign
def language(x):
    if x == 'English':
        return 'English'
    else:
        return 'Foreign'
    
# Lets apply the function on the language column
data['language'] = data['language'].apply(language)
    
# Lets check the values again
data['language'].value_counts()

English    3673
Foreign     180
Name: language, dtype: int64

In [19]:
data['duration'].value_counts()

101.0    113
100.0    109
98.0     105
99.0      97
90.0      97
        ... 
216.0      1
240.0      1
65.0       1
45.0       1
225.0      1
Name: duration, Length: 157, dtype: int64

In [20]:
# The duration of movies is not varrying a lot but we know that most of the users either like watching long movies or short
# duration movies. we can categories that movie in two part i.e short and long

# Lets define a function for categorizing duration of movies
def duration(x):
    if x <= 120:
        return 'short'
    else:
        return 'long'

# Lets apply this function on the durtion column 
data['duration'] = data['duration'].apply(duration)

# Lets check the value of duration column
data['duration'].value_counts()

short    2936
long      917
Name: duration, dtype: int64

In [21]:
# Lets also check the values in the genres column

data['genres'].value_counts()

Drama                                              153
Comedy|Drama|Romance                               151
Comedy|Drama                                       147
Comedy                                             145
Comedy|Romance                                     135
                                                  ... 
Comedy|Romance|Sci-Fi|Thriller                       1
Action|Adventure|Family|Fantasy|Sci-Fi|Thriller      1
Adventure|Comedy|Drama|Music                         1
Action|Drama|Fantasy|Mystery|Sci-Fi|Thriller         1
Adventure|Comedy|Horror                              1
Name: genres, Length: 762, dtype: int64

In [22]:
data['genres'].str.split('|')[0]

['Action', 'Adventure', 'Fantasy', 'Sci-Fi']

In [24]:
# So we can see from the above output that most of the movies are having a lot of genres
# also, a movie can have so many genres so lets keep four genres

data['Moviegenres'] = data['genres'].str.split('|')
data['Genre1'] = data['Moviegenres'].apply(lambda x: x[0])

# some of the movies have only one genre. In such cases, assign the same genre to 'genre_2' as well
data['Genre2'] = data['Moviegenres'].apply(lambda x: x[1] if len(x) > 1 else x[0])
data['Genre3'] = data['Moviegenres'].apply(lambda x: x[2] if len(x) > 2 else x[0])
data['Genre4'] = data['Moviegenres'].apply(lambda x: x[3] if len(x) > 3 else x[0])

# Lets check the head of the data
data[['genres','Genre1','Genre2','Genre3','Genre4']].head(5)


,genres,Genre1,Genre2,Genre3,Genre4
0,Action|Adventure|Fantasy|Sci-Fi,Action,Adventure,Fantasy,Sci-Fi
1,Action|Adventure|Fantasy,Action,Adventure,Fantasy,Action
2,Action|Adventure|Thriller,Action,Adventure,Thriller,Action
3,Action|Thriller,Action,Thriller,Action,Action
5,Action|Adventure|Sci-Fi,Action,Adventure,Sci-Fi,Action
